In [1]:
import pandas as pd
import mysql
import mysql.connector
import csv


In [2]:
def mitarbeiter_func():
    #Import Mitarbeiter Tabelle
    mitarbeiter = pd.read_excel('mitarbeiter.xlsx') 

    #Spaltung Ort und PLZ
    mitarbeiter.insert(3,"PLZ", mitarbeiter.Ort.str.split(" ").str[0].tolist())
    mitarbeiter["Ort"] = mitarbeiter.Ort.str.split(" ").str[1].tolist()

    #Abteilung in AbteilungsNR tauschen
    mitarbeiter.loc[mitarbeiter["Abteilung"] == "IT","Abteilung"] = 2
    mitarbeiter.loc[mitarbeiter["Abteilung"] == "Consulting","Abteilung"] = 4
    mitarbeiter.loc[mitarbeiter["Abteilung"] == "Sales","Abteilung"] = 0
    mitarbeiter.loc[mitarbeiter["Abteilung"] == "Marketing","Abteilung"] = 3
    mitarbeiter.loc[mitarbeiter["Abteilung"] == "Controlling","Abteilung"] = 1
   
    #Spaltung in Vorname und Name
    mitarbeiter.insert(1,"Vorname", mitarbeiter.Name.str.split(" ").str[0].tolist())
    mitarbeiter["Name"] = mitarbeiter.Name.str.split(" ").str[1].tolist()

    #Typanpassung
    mitarbeiter.infer_objects()

    #Export der Datei
    mitarbeiter_csv_data = mitarbeiter.to_csv('ETL Daten/mitarbeiter_etl.csv', header=False, index=False) 

In [3]:
def projekt_func():
    #Import Projekt Tabelle
    projekt = pd.read_excel('projekt.xlsx') 
    #Import Mitarbeiter Tabelle
    mitarbeiter = pd.read_excel('mitarbeiter.xlsx') 

    #Projektleiter Fremdschlüssel hinzufügen
    for i in range(len(projekt)):
        for j in range(len(mitarbeiter)):
            if projekt["Leiter"][i] == (mitarbeiter["Name"][j]):
                projekt["Leiter"][i] = mitarbeiter["PerNR"][j]
                break
    
    #Typanpassung
    projekt.infer_objects()
                
    #Export der Datei
    projekt_csv_data = projekt.to_csv('ETL Daten/projekt_etl.csv', header=False, index=False) 

In [4]:
def abteilung_func():
    #Import Abteilung Tabelle
    abteilung = pd.read_excel('Abteilung.xlsx') 
    #Import Mitarbeiter Tabelle
    mitarbeiter = pd.read_excel('mitarbeiter.xlsx') 

    #Abteilungsleiter Fremdschlüssel hinzufügen
    for i in range(len(abteilung)):
        for j in range(len(mitarbeiter)):
            if abteilung["Abteilungsleiter"][i] == (mitarbeiter["Name"][j]):
                abteilung["Abteilungsleiter"][i] = mitarbeiter["PerNR"][j]
    
    #Typanpassung
    abteilung.infer_objects()

    #Export der Datei
    abteilung_csv_data = abteilung.to_csv('ETL Daten/Abteilung_etl.csv', header=False, index=False) 

In [5]:
def projektzeiten_func():
    #Import Projektzeiten Tabelle
    projektzeiten = pd.read_excel('projektzeiten.xlsx')
    #Import Mitarbeiter Tabelle
    mitarbeiter = pd.read_excel('mitarbeiter.xlsx') 

    #Abteilungsleiter Fremdschlüssel hinzufügen
    for i in range(len(projektzeiten)):
        for j in range(len(mitarbeiter)):
            if projektzeiten["Mitarbeiter"][i] == (mitarbeiter["Name"][j]):
                projektzeiten["Mitarbeiter"][i] = mitarbeiter["PerNR"][j]
                break
    
    #Typanpassung
    projektzeiten.infer_objects()

    #Export der Datei
    projektzeiten_csv_data = projektzeiten.to_csv('ETL Daten/projektzeiten_etl.csv', header=False, index=False) 

In [6]:

mitarbeiter_func()
projekt_func()
abteilung_func()
projektzeiten_func()


<ipython-input-3-d305f974073e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projekt["Leiter"][i] = mitarbeiter["PerNR"][j]


In [7]:
mydb = mysql.connector.connect(host = "127.0.0.1", user = "root", passwd = "", db = "")

In [8]:

cursor = mydb.cursor()
cursor.execute("SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0")

cursor.execute("SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0")
cursor.execute("SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION'")

cursor.execute("DROP SCHEMA IF EXISTS `Firma` ")
cursor.execute("CREATE SCHEMA IF NOT EXISTS `Firma` DEFAULT CHARACTER SET utf8 ")
cursor.execute("USE `Firma` ")

cursor.execute("DROP TABLE IF EXISTS `Firma`.`Abteilung` ")
cursor.execute("CREATE TABLE IF NOT EXISTS `Firma`.`Abteilung` ( `AbtNR` INT ZEROFILL UNSIGNED NOT NULL, `Bezeichnung` VARCHAR(45) NOT NULL, `Abteilungsleiter` INT ZEROFILL UNSIGNED NULL, `Etat` FLOAT(10,2) NOT NULL, PRIMARY KEY (`AbtNR`), UNIQUE INDEX `Bezeichnung_UNIQUE` (`Bezeichnung` ASC), UNIQUE INDEX `Abteilungsleiter_UNIQUE` (`Abteilungsleiter` ASC) ) ENGINE = InnoDB")

cursor.execute("DROP TABLE IF EXISTS `Firma`.`Mitarbeiter` ")
cursor.execute("CREATE TABLE IF NOT EXISTS `Firma`.`Mitarbeiter` ( `PerNR` INT UNSIGNED ZEROFILL NOT NULL, `Name` VARCHAR(45) NOT NULL, `Vorname` VARCHAR(45) NOT NULL, `Strasse` VARCHAR(45) NOT NULL, `PLZ` INT(5) NOT NULL, `Ort` VARCHAR(45) NOT NULL, `Telefon` INT NOT NULL, `Abteilung` INT NOT NULL, `Geschlecht` ENUM('m', 'w') NOT NULL, `eingestellt` DATE NOT NULL, `KFZ1` VARCHAR(10) NULL, `KFZ2` VARCHAR(10) NULL, PRIMARY KEY (`PerNR`)) ENGINE = InnoDB ")

cursor.execute("DROP TABLE IF EXISTS `Firma`.`Projekt` ")
cursor.execute("CREATE TABLE IF NOT EXISTS `Firma`.`Projekt` ( `ProNR` INT NOT NULL, `Bezeichnung` VARCHAR(45) NOT NULL, `Start` DATE NOT NULL, `Ende` DATE NULL, `Auftragswert` FLOAT(12,2) NOT NULL, `bezahlt` FLOAT(12,2) NULL, `Leiter` INT NOT NULL, PRIMARY KEY (`ProNR`)) ENGINE = InnoDB ")

cursor.execute("DROP TABLE IF EXISTS `Firma`.`ProjektMitarbeiter` ")
cursor.execute("CREATE TABLE IF NOT EXISTS `Firma`.`ProjektMitarbeiter` (`Projekt_ProNR` INT NOT NULL, `Mitarbeiter_PerNR` INT NOT NULL, `Zeitanteil` FLOAT(10,1) NOT NULL, PRIMARY KEY (`Projekt_ProNR`, `Mitarbeiter_PerNR`)) ENGINE = InnoDB")

cursor.execute("SET SQL_MODE=@OLD_SQL_MODE")
cursor.execute("SET FOREIGN_KEY_CHECKS=@OLD_FOREIGN_KEY_CHECKS")
cursor.execute("SET UNIQUE_CHECKS=@OLD_UNIQUE_CHECKS")

cursor.close()

True

In [9]:
cursor = mydb.cursor()
cursor.execute("Use Firma")
cursor.execute("LOAD DATA INFILE 'ETL_Project/ETL Daten/mitarbeiter_etl.csv' INTO TABLE mitarbeiter  FIELDS TERMINATED BY ','")
cursor.execute("LOAD DATA INFILE 'ETL_Project/ETL Daten/Abteilung_etl.csv' INTO TABLE abteilung  FIELDS TERMINATED BY ','")
cursor.execute("LOAD DATA INFILE 'ETL_Project/ETL Daten/projekt_etl.csv' INTO TABLE projekt  FIELDS TERMINATED BY ','")
cursor.execute("LOAD DATA INFILE 'ETL_Project/ETL Daten/projektzeiten_etl.csv' INTO TABLE projektmitarbeiter  FIELDS TERMINATED BY ','")

cursor.close()

True

In [10]:
mydb.commit()
mydb.close()